# Nonlinear Binary Time-Series Models (Logit/Probit) — Macro (Peru-like, Simulated)
This notebook simulates monthly macroeconomic data resembling a LATAM economy and estimates:
- Dynamic **Probit** and **Logit** for recession probabilities
- **Threshold-Logit** (commodity regimes)
- **Heteroskedastic Probit** (multiplicative heteroskedasticity MLE)
- **FX Crisis** early-warning **Logit** on EPI-defined events

It also reports validation metrics (Brier, Log score, AUC), calibration plots, ROC curves, and Ljung–Box tests.


In [1]:
import os, math, warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from scipy import stats, optimize
import statsmodels.api as sm
from statsmodels.tools.tools import add_constant
from statsmodels.stats.diagnostic import acorr_ljungbox

from sklearn.metrics import roc_auc_score, roc_curve, brier_score_loss, log_loss
from sklearn.calibration import calibration_curve

OUT_FIGS = "output/figs"
OUT_TBLS = "output/tables"
os.makedirs(OUT_FIGS, exist_ok=True)
os.makedirs(OUT_TBLS, exist_ok=True)


### 1) Simulate monthly macro data (Peru-like)

In [2]:
def simulate_macro(T=240, seed=2025):
    rng = np.random.default_rng(seed)
    idx = pd.date_range("2005-01-01", periods=T, freq="MS")
    h = np.zeros(T)
    h[0] = -1.5
    for t in range(1, T):
        h[t] = -1.2 + 0.90*h[t-1] + 0.15*rng.standard_normal()
    vol_pi = np.exp(0.5*h)

    comm = np.zeros(T); comm[0] = 100.0
    for t in range(1, T):
        comm[t] = 100 + 0.95*(comm[t-1]-100) + 1.2*np.sin(2*np.pi*t/48) + 0.8*rng.standard_normal()

    y_gap = np.zeros(T)
    for t in range(1, T):
        y_gap[t] = 0.8*y_gap[t-1] + 0.02*(comm[t]-100) - 0.10 + 0.2*rng.standard_normal()

    pi = np.zeros(T)
    for t in range(1, T):
        pi[t] = 0.7*pi[t-1] + 0.03*(comm[t]-100)/10 + 3.0 + vol_pi[t]*rng.standard_normal()

    i_us = np.zeros(T)
    for t in range(1, T):
        i_us[t] = 0.98*i_us[t-1] + 0.02*2.0 + 0.05*rng.standard_normal()

    i_policy = np.zeros(T)
    for t in range(1, T):
        i_policy[t] = (0.7*i_policy[t-1]
                       + 0.3*(2.5 + 0.5*(pi[t]-3.0) + 0.4*y_gap[t])
                       + 0.05*rng.standard_normal())

    slope = 1.5 + 0.3*y_gap + 0.1*rng.standard_normal(T) - 0.2*(pi - 3.0)/10
    pmi = 50 + 5*y_gap + 0.5*rng.standard_normal(T) - 0.2*(pi - 3.0)
    embi = 2.0 + 0.8*(-y_gap) + 0.2*(pi-3.0) + 0.3*rng.standard_normal(T)
    embi = np.maximum(embi, 0.5)

    dlog_fx = 0.002*(-y_gap) + 0.001*(pi-3.0) + 0.01*rng.standard_normal(T)
    dlog_res = -0.003*(-y_gap) + 0.002*rng.standard_normal(T)

    df = pd.DataFrame({
        "y_gap": y_gap,
        "pi": pi,
        "comm": comm,
        "i_us": i_us,
        "i_policy": i_policy,
        "slope": slope,
        "pmi": pmi,
        "embi": embi,
        "dlog_fx": dlog_fx,
        "dlog_res": dlog_res
    }, index=idx)

    z_fx = (df["dlog_fx"] - df["dlog_fx"].mean())/df["dlog_fx"].std()
    z_res = (df["dlog_res"] - df["dlog_res"].mean())/df["dlog_res"].std()
    df["EPI"] = z_fx - z_res
    return df

df = simulate_macro()
eta_rec = -1.0 - 1.5*df["y_gap"].values - 0.6*df["slope"].values + 0.5*df["embi"].values
p_rec_true = stats.norm.cdf(eta_rec)
rng = np.random.default_rng(2026)
df["y_rec"] = rng.binomial(1, np.clip(p_rec_true, 0.001, 0.999))
tau_epi = np.quantile(df["EPI"], 0.85)
df["y_fx"] = (df["EPI"] > tau_epi).astype(int)
pi_12m = df["pi"].rolling(12, min_periods=12).mean()
tau_pi = np.nanquantile(pi_12m, 0.80)
df["y_hi"] = (pi_12m > tau_pi).astype(int).fillna(0)
df["pi_12m"] = pi_12m
df.describe().T.to_csv(os.path.join(OUT_TBLS, "01_descriptives.csv"))
df.head()


,y_gap,pi,comm,i_us,i_policy,slope,pmi,embi,dlog_fx,dlog_res,EPI,y_rec,y_fx,y_hi,pi_12m
2005-01-01,0.000000,0.000000,100.000000,0.000000,0.000000,1.645447,50.556308,1.134289,-0.024312,0.002325,-4.304747,0,0,0,NaN
2005-02-01,0.105973,3.275552,99.904631,0.090527,0.829413,1.567257,50.044484,1.557588,0.016451,0.005051,-1.246652,0,0,0,NaN
2005-03-01,-0.001503,5.204726,101.499111,0.144430,1.623782,1.322321,50.111506,2.409125,0.023384,-0.001959,2.087966,0,1,0,NaN
2005-04-01,-0.289532,6.703919,103.206089,0.182325,2.353954,1.391350,47.269714,3.157416,0.006267,-0.003064,0.788797,1,0,0,NaN
2005-05-01,-0.014528,7.698998,103.843249,0.185081,3.057131,1.528135,48.418520,2.865510,0.006538,0.004709,-2.111277,0,0,0,NaN


### 2) Dynamic Probit & Logit for Recession

In [3]:
def time_split(df, frac_train=0.75):
    T = len(df); T_tr = int(frac_train*T)
    return df.index[:T_tr], df.index[T_tr:]

dfA = df.copy()
dfA["y_rec_l1"] = dfA["y_rec"].shift(1)
X_cols = ["y_rec_l1","y_gap","slope","embi","pmi"]
dfA_model = dfA[X_cols+["y_rec"]].dropna()

X = add_constant(dfA_model[X_cols])
y = dfA_model["y_rec"].values
tr_idx, te_idx = time_split(dfA_model, 0.75)

X_tr, X_te = X.loc[tr_idx], X.loc[te_idx]
y_tr, y_te = y[:len(X_tr)], y[len(X_tr):]

probit_res = sm.Probit(y_tr, X_tr).fit(disp=False, cov_type="HC1")
logit_res  = sm.Logit(y_tr,  X_tr).fit(disp=False, cov_type="HC1")

p_probit_te = probit_res.predict(X_te)
p_logit_te  = logit_res.predict(X_te)

def brier(y, p): return brier_score_loss(y, p)
def log_score(y, p, eps=1e-12):
    p = np.clip(p, eps, 1-eps)
    return -np.mean(y*np.log(p)+(1-y)*np.log(1-p))

metrics = pd.DataFrame([
    {"model":"Probit", "AUC": roc_auc_score(y_te, p_probit_te),
     "Brier": brier(y_te, p_probit_te), "LogScore": log_score(y_te, p_probit_te)},
    {"model":"Logit",  "AUC": roc_auc_score(y_te, p_logit_te),
     "Brier": brier(y_te, p_logit_te),  "LogScore": log_score(y_te, p_logit_te)},
])
metrics.to_csv(os.path.join(OUT_TBLS, "02_metrics_rec_test.csv"), index=False)

print(probit_res.summary())
print(logit_res.summary())
metrics


                          Probit Regression Results                           
Dep. Variable:                      y   No. Observations:                  179
Model:                         Probit   Df Residuals:                      173
Method:                           MLE   Df Model:                            5
Date:                Tue, 16 Sep 2025   Pseudo R-squ.:                  0.3869
Time:                        01:14:41   Log-Likelihood:                -75.318
converged:                       True   LL-Null:                       -122.84
Covariance Type:                  HC1   LLR p-value:                 5.852e-19
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          2.6955     11.778      0.229      0.819     -20.389      25.780
y_rec_l1      -0.0862      0.248     -0.347      0.728      -0.573       0.400
y_gap         -1.6752      1.322     -1.268      0.2

,model,AUC,Brier,LogScore
0,Probit,0.936080,0.093254,0.282129
1,Logit,0.934659,0.093857,0.286549


### 3) Validation plots (calibration, ROC, probability paths)

In [4]:
def calibration_plot(y_true, p_hat, title, path_png):
    prob_true, prob_pred = calibration_curve(y_true, p_hat, n_bins=10, strategy="uniform")
    plt.figure(figsize=(6,4))
    plt.plot(prob_pred, prob_true, marker="o", linestyle="-")
    plt.plot([0,1],[0,1], linestyle="--")
    plt.title(title); plt.xlabel("Predicted probability"); plt.ylabel("Observed frequency")
    plt.tight_layout(); plt.savefig(path_png, dpi=160); plt.close()

def roc_plot(y_true, p_hat, title, path_png):
    fpr, tpr, _ = roc_curve(y_true, p_hat)
    auc = roc_auc_score(y_true, p_hat)
    plt.figure(figsize=(6,4))
    plt.plot(fpr, tpr, linestyle="-")
    plt.plot([0,1],[0,1], linestyle="--")
    plt.title(f"{title} (AUC={auc:.3f})"); plt.xlabel("False Positive Rate"); plt.ylabel("True Positive Rate")
    plt.tight_layout(); plt.savefig(path_png, dpi=160); plt.close()

def prob_plot(idx, p_hat, title, path_png):
    plt.figure(figsize=(9,3.2))
    plt.plot(idx, p_hat, linestyle="-")
    plt.ylim(0,1); plt.title(title); plt.xlabel("Time"); plt.ylabel("Probability")
    plt.tight_layout(); plt.savefig(path_png, dpi=160); plt.close()

calibration_plot(y_te, p_probit_te, "Calibration: Dynamic Probit (Recession) – Test",
                 os.path.join(OUT_FIGS, "calib_probit_rec.png"))
roc_plot(y_te, p_probit_te, "ROC: Dynamic Probit (Recession) – Test",
         os.path.join(OUT_FIGS, "roc_probit_rec.png"))
prob_plot(X_te.index, p_probit_te, "Predicted Probabilities – Probit (Test)",
          os.path.join(OUT_FIGS, "probs_probit_rec.png"))


### 4) Threshold-Logit (commodity threshold with interactions)

In [6]:
c = np.quantile(dfA_model.reindex(X.index)["comm"], 0.40)
reg_low = (dfA_model.reindex(X.index)["comm"] <= c).astype(int)

X_thr = X.copy()
for col in ["y_rec_l1","y_gap","slope","embi","pmi"]:
    X_thr[f"{col}:low"] = X_thr[col] * reg_low.values

logit_thr = sm.Logit(y_tr, X_thr.loc[tr_idx]).fit(disp=False, cov_type="HC1")
p_thr_te  = logit_thr.predict(X_thr.loc[te_idx])

pd.DataFrame([{"model":"Threshold-Logit",
               "AUC": roc_auc_score(y_te, p_thr_te),
               "Brier": brier(y_te, p_thr_te),
               "LogScore": log_score(y_te, p_thr_te)}]).to_csv(
    os.path.join(OUT_TBLS, "05_metrics_threshold_logit_rec.csv"), index=False
)

print(logit_thr.summary())


KeyError: 'comm'

### 5) Heteroskedastic Probit (multiplicative heteroskedasticity MLE)

In [7]:
dfH = dfA_model[["y_rec","y_rec_l1","y_gap","slope","embi"]].copy()
dfH["vol_proxy"] = df.reindex(dfH.index)["embi"].values

XH = add_constant(dfH[["y_rec_l1","y_gap","slope","embi"]]).values
ZH = add_constant(dfH[["vol_proxy"]]).values
yH = dfH["y_rec"].values

def neg_loglik_het_probit(theta):
    kx = XH.shape[1]; kz = ZH.shape[1]
    beta = theta[:kx]; delta = theta[kx:kx+kz]
    eta = XH @ beta; sigma = np.exp(ZH @ delta)
    p = stats.norm.cdf(eta / sigma); p = np.clip(p, 1e-9, 1-1e-9)
    return -np.sum(yH*np.log(p) + (1-yH)*np.log(1-p))

init = sm.Probit(yH, XH).fit(disp=False)
theta0 = np.r_[np.asarray(init.params), np.array([0.0, 0.05])]

opt = optimize.minimize(neg_loglik_het_probit, theta0, method="BFGS")
theta_hat = opt.x; se_hat = np.sqrt(np.diag(opt.hess_inv))

par_names = [f"beta_{n}" for n in ["const","y_rec_l1","y_gap","slope","embi"]] +             [f"delta_{n}" for n in ["const","vol_proxy"]]
tab = pd.DataFrame({"param": par_names, "estimate": theta_hat, "std_error": se_hat})
tab.to_csv(os.path.join(OUT_TBLS, "06_params_heteroskedastic_probit.csv"), index=False)
tab


,param,estimate,std_error
0,beta_const,-3.248971,1.009234
1,beta_y_rec_l1,-0.067485,0.224858
2,beta_y_gap,-2.279348,0.886970
3,beta_slope,0.564645,0.810972
4,beta_embi,0.702014,0.321993
5,delta_const,-0.860117,0.945429
6,delta_vol_proxy,0.233546,0.243762


### 6) FX Crisis Early-Warning Logit (EPI-defined event)

In [8]:
dfE = df.copy(); dfE["EPI_l1"] = dfE["EPI"].shift(1); dfE = dfE.dropna()
yE = dfE["y_fx"].values
XE = add_constant(dfE[["embi","slope","comm","y_gap","EPI_l1"]])

T = len(XE); T_tr = int(0.75*T)
XE_tr, XE_te = XE.iloc[:T_tr], XE.iloc[T_tr:]
yE_tr, yE_te = yE[:T_tr], yE[T_tr:]

logit_fx = sm.Logit(yE_tr, XE_tr).fit(disp=False, cov_type="HC1")
p_fx_te = logit_fx.predict(XE_te)

print(logit_fx.summary())

pd.DataFrame([{"model":"Logit – FX Crisis (excl. defining components)",
               "AUC": roc_auc_score(yE_te, p_fx_te),
               "Brier": brier_score_loss(yE_te, p_fx_te),
               "LogScore": log_loss(yE_te, np.clip(p_fx_te,1e-12,1-1e-12))}]).to_csv(
    os.path.join(OUT_TBLS, "07_metrics_logit_fx.csv"), index=False
)


                           Logit Regression Results                           
Dep. Variable:                      y   No. Observations:                  171
Model:                          Logit   Df Residuals:                      165
Method:                           MLE   Df Model:                            5
Date:                Tue, 16 Sep 2025   Pseudo R-squ.:                  0.1418
Time:                        01:15:38   Log-Likelihood:                -52.952
converged:                       True   LL-Null:                       -61.701
Covariance Type:                  HC1   LLR p-value:                  0.003646
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -3.8516      7.071     -0.545      0.586     -17.711      10.008
embi           0.7420      0.760      0.976      0.329      -0.748       2.232
slope         -2.8982      2.034     -1.425      0.1